In [ ]:
pip install PyMuPDF fitz

In [8]:
#Importing Libraries
filepath = '/content/Profile.pdf'
from operator import itemgetter
import fitz
import json

In [9]:
# To identify fonts
def fonts(doc, granularity=False):
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles


# To identify font tags
def font_tags(font_counts, styles):
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  

    # sorting the font sizes high to low, so that we can append the right integer to each tag
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h>'
        elif size < p_size:
            size_tag[size] = '<s>'

    return size_tag


def headers_para(doc, size_tag):
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para


def main():

    document = filepath
    doc = fitz.open(document)

    font_counts, styles = fonts(doc, granularity=False)

    size_tag = font_tags(font_counts, styles)

    elements = headers_para(doc, size_tag)

    return elements


if __name__ == '__main__':
    text=main()

Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


In [10]:
# Function to convert  
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [11]:
text = listToString(text)
print(text)

|| <h>Contact|  <s>www.linkedin.com/in/adwa-| altami-58b2bb188  (LinkedIn)|  <h>Adwa AlTami|  <p>Business Analyst Intern at HRcom.io| Riyadh|  <h>Experience|  <p>E-Commerce Club | ةينورتكلإلا ةراجتلا يدان|  <s>1 year 1 month|  <s>President| <s>September 2021 - Present (5 months)|  <s>Vice President| <s>January 2021 - September 2021 (9 months)|  <p>HRcom.io| <s>Business Analyst Intern| <s>May 2021 - Present (9 months)| Riyadh, Saudi Arabia|  <p>Prince Sultan University - Data Science Club| <s>Graphic Designer| <s>February 2020 - May 2021 (1 year 4 months)|  <p>Prince Sultan University - Accounting Club| <s>Graphic Designer| <s>January 2020 - May 2021 (1 year 5 months)|  <h>Education|  <p>Prince Sultan University|  <s>Information Systems and E-Commerce · (September 2019 - September 2024)|  <p>AlAfaq Schools|  <s>American Diploma  · (2019)| | <s>Page 1 of 1|
